In [1]:
from math import ceil, pi, e
from termcolor import colored
import numpy as np

In [2]:
def checkint(floatt):
  floatt = float(floatt)
  if floatt%1!=0:
    return floatt
  else:
    return int(floatt)

def truncate(floatt,n):
  floatt = str(float(floatt))
  indx = floatt.find('.')
  if indx!=-1:
    try:
      return checkint(floatt[:indx+n+1])
    except:
      return checkint(floatt)
  else:
    return checkint(floatt)

def printt(string,color="red",end='\n'):
  print(colored(string,color),end=end)

In [3]:
def powermethod(A,x0,k,n):
  printt(f'(A-q{k}I)inverse * X{k} =')
  dot = A.dot(np.array([x0]).transpose())
  #dot = [truncate(i,n) for i in dot]
  u = max(dot, key=abs)
  dotdiv = np.divide(dot,u)
  #dotdiv = [truncate(i,n) for i in dotdiv]
  print(f'{A} *')
  print(f'{np.array([x0]).transpose()} =\n')
  print(f'{dot} =\n')
  print(f'{u} *')
  print(f'{dotdiv} = u{k+1}X{k+1}')

  return truncate(u[0],n), dotdiv.flatten()

In [4]:
def finderror(lmbdas,i,n):
  a1, a2 = lmbdas[i-1], lmbdas[i]
  if i==0:
    a1 = 0
  error = truncate(abs((a2-a1)/a2),n)
  print(f'Error = | ({a2} - {a1})/{a2} | => {error}\n')
  return error

def dominantinversepowermethod(A,x0,errorlim,n,numiters):
  flag = errorlim!=None
  A = np.array(A)
  lmbdas = list()  
  printt(f'q = (Xt *\n    A *\n    X) /\n    (Xt *\n    X)')

  for i in range(numiters):    
    x0 = np.array([x0])
    x0t = x0.transpose()
    num = x0.dot(A).dot(x0t)[0][0]
    den = x0.dot(x0t)[0][0]
    q = truncate(num/den,n)
    print(f"q{i+1} =\n{colored('(','blue')}{x0} *\n{A} *\n{x0t}{colored(')','blue')} /\n{colored('(','blue')}{x0} *\n{x0t}{colored(')','blue')}")
    printt(f"\nq{i+1} = {q}",color='green')
    Amin = A - q*np.identity(len(A), dtype=int) # A - qI
    Ainv = np.linalg.inv(Amin)    # (A - qI)inverse
    print(f'\n(A - q{i}I) =\n{Amin}')
    print(f'\n(A - q{i}I)inverse =\n{Ainv}')
    printt(f'Now find maximum eigen value of (A-q{i}I)inverse using power method\n')

    micro, x0 = powermethod(Ainv,x0[0],i,n)
    lmbdas.append(truncate((1/micro)+q,n))
    printt(f'u{i+1} = {micro}',color='green')
    print(f"λ{i+1} = (1/u{i+1}) + q{i+1} {colored(f'= {truncate(1/micro,n)} + {q}','blue')} {colored(f'= {lmbdas[i]}','blue')}")
    error = finderror(lmbdas,i,n)
    if flag:
      if error<errorlim:
        break
    printt('-------------------------------------------------------------------------------------------------------------\n')

In [5]:
def smallesteigenvalue(A,x0,errorlim,n,numiters):
  flag = errorlim!=None
  rows = len(A)
  A = np.array(A)
  x0 = np.array([x0]).transpose()
  lmbdas = list()
  B = np.linalg.inv(A)
  for i in range(rows):
    if i==0:print(f'A = {A[i]}')
    else:print(f'    {A[i]}')  
  for i in range(rows):
    if i==0:print(f'B = Ainverse = {B[i]}')
    else:print(f'              {B[i]}')  
  for k in range(numiters):
    Bx = B.dot(x0)
    u = max(Bx, key=abs)
    dotdiv = np.divide(Bx,u)
    for i in range(rows):
      if i==0:print(f'\nBx{k} = {B[i]} * {x0[i]}')
      else:print(f'      {B[i]}   {x0[i]}')
    for i in range(rows):
      if i==0:print(f'\n    = {Bx[i]} = {u}{dotdiv[i]} = u{k+1}x{k+1}')
      else:print(f'      {Bx[i]}        {dotdiv[i]}')
    x0 = dotdiv
    lmb = 1/u
    printt(f'λ{k} = 1/u{k} = 1/{u} =>{lmb}',color='blue')
    lmbdas.append(lmb)
    error = finderror(lmbdas,k,n)
    if flag:
      if error<errorlim:
        break
    printt('-------------------------------------')

DOMINANT EIGEN VALUE

In [6]:
A = [[-4,14,0],         # matrix A
     [-5,13,0],
     [-1,0,2]]

x0 = [1,1,1]            # initial x0 transpose
errorlim = None       # if errorlim, else set to None
n = 4                 # for truncation (n decimal places)
numiters = 1          # num of iterations (set to 50 if to find using errorlim)

np.set_printoptions(precision = n)
dominantinversepowermethod(A,x0,errorlim,n,numiters)

q = (Xt *
    A *
    X) /
    (Xt *
    X)
q1 =
([[1 1 1]] *
[[-4 14  0]
 [-5 13  0]
 [-1  0  2]] *
[[1]
 [1]
 [1]]) /
([[1 1 1]] *
[[1]
 [1]
 [1]])

q1 = 6.3333

(A - q0I) =
[[-10.3333  14.       0.    ]
 [ -5.       6.6667   0.    ]
 [ -1.       0.      -4.3333]]

(A - q0I)inverse =
[[  6.0007 -12.6014  -0.    ]
 [  4.5005  -9.301   -0.    ]
 [ -1.3848   2.908   -0.2308]]
Now find maximum eigen value of (A-q0I)inverse using power method

(A-q0I)inverse * X0 =
[[  6.0007 -12.6014  -0.    ]
 [  4.5005  -9.301   -0.    ]
 [ -1.3848   2.908   -0.2308]] *
[[1]
 [1]
 [1]] =

[[-6.6007]
 [-4.8005]
 [ 1.2925]] =

[-6.6007] *
[[ 1.    ]
 [ 0.7273]
 [-0.1958]] = u1X1
u1 = -6.6006
λ1 = (1/u1) + q1 = -0.1515 + 6.3333 = 6.1817
Error = | (6.1817 - 0)/6.1817 | => 1

-------------------------------------------------------------------------------------------------------------



In [7]:
A = [[1,1,0,0],         # matrix A
     [1,2,0,1],
     [0,0,3,3],
     [0,1,3,2]]

x0 = [1,1,0,1]            # initial x0 transpose
errorlim = None       # if errorlim, else set to None
n = 4                 # for truncation (n decimal places)
numiters = 3          # num of iterations (set to 50 if to find using errorlim)

np.set_printoptions(precision = n)
dominantinversepowermethod(A,x0,errorlim,n,numiters)

q = (Xt *
    A *
    X) /
    (Xt *
    X)
q1 =
([[1 1 0 1]] *
[[1 1 0 0]
 [1 2 0 1]
 [0 0 3 3]
 [0 1 3 2]] *
[[1]
 [1]
 [0]
 [1]]) /
([[1 1 0 1]] *
[[1]
 [1]
 [0]
 [1]])

q1 = 3

(A - q0I) =
[[-2  1  0  0]
 [ 1 -1  0  1]
 [ 0  0  0  3]
 [ 0  1  3 -1]]

(A - q0I)inverse =
[[-1.     -1.      0.3333 -0.    ]
 [-1.     -2.      0.6667  0.    ]
 [ 0.3333  0.6667 -0.1111  0.3333]
 [ 0.      0.      0.3333  0.    ]]
Now find maximum eigen value of (A-q0I)inverse using power method

(A-q0I)inverse * X0 =
[[-1.     -1.      0.3333 -0.    ]
 [-1.     -2.      0.6667  0.    ]
 [ 0.3333  0.6667 -0.1111  0.3333]
 [ 0.      0.      0.3333  0.    ]] *
[[1]
 [1]
 [0]
 [1]] =

[[-2.    ]
 [-3.    ]
 [ 1.3333]
 [ 0.    ]] =

[-3.] *
[[ 0.6667]
 [ 1.    ]
 [-0.4444]
 [-0.    ]] = u1X1
u1 = -3
λ1 = (1/u1) + q1 = -0.3333 + 3 = 2.6666
Error = | (2.6666 - 0)/2.6666 | => 1

-------------------------------------------------------------------------------------------------------------

q2 =
([[ 0.6667  1.     

SMALLEST EIGEN VALUE

In [8]:
A = [[-2,-3],         # matrix A
     [6,7]]

x0 = [1,1]            # initial x0 transpose
errorlim = None       # if errorlim, else set to None
n = 4                 # for truncation (n decimal places)
numiters = 2          # num of iterations (set to 50 if to find using errorlim)

np.set_printoptions(precision = n)
smallesteigenvalue(A,x0,errorlim,n,numiters)

A = [-2 -3]
    [6 7]
B = Ainverse = [1.75 0.75]
              [-1.5 -0.5]

Bx0 = [1.75 0.75] * [1]
      [-1.5 -0.5]   [1]

    = [2.5] = [2.5][1.] = u1x1
      [-2.]        [-0.8]
λ0 = 1/u0 = 1/[2.5] =>[0.4]
Error = | ([0.4] - 0)/[0.4] | => 1

-------------------------------------

Bx1 = [1.75 0.75] * [1.]
      [-1.5 -0.5]   [-0.8]

    = [1.15] = [1.15][1.] = u2x2
      [-1.1]        [-0.9565]
λ1 = 1/u1 = 1/[1.15] =>[0.8696]
Error = | ([0.8696] - [0.4])/[0.8696] | => 0.54

-------------------------------------


In [9]:
A = [[1,1,0,0],         # matrix A
     [1,2,0,1],
     [0,0,3,3],
     [0,1,3,2]]

x0 = [1,1,0,1]            # initial x0 transpose
errorlim = None       # if errorlim, else set to None
n = 4                 # for truncation (n decimal places)
numiters = 3          # num of iterations (set to 50 if to find using errorlim)

np.set_printoptions(precision = n)
smallesteigenvalue(A,x0,errorlim,n,numiters)

A = [1 1 0 0]
    [1 2 0 1]
    [0 0 3 3]
    [0 1 3 2]
B = Ainverse = [ 1.5 -0.5  0.5 -0.5]
              [-0.5  0.5 -0.5  0.5]
              [ 0.5    -0.5    -0.1667  0.5   ]
              [-0.5  0.5  0.5 -0.5]

Bx0 = [ 1.5 -0.5  0.5 -0.5] * [1]
      [-0.5  0.5 -0.5  0.5]   [1]
      [ 0.5    -0.5    -0.1667  0.5   ]   [0]
      [-0.5  0.5  0.5 -0.5]   [1]

    = [0.5] = [0.5][1.] = u1x1
      [0.5]        [1.]
      [0.5]        [1.]
      [-0.5]        [-1.]
λ0 = 1/u0 = 1/[0.5] =>[2.]
Error = | ([2.] - 0)/[2.] | => 1

-------------------------------------

Bx1 = [ 1.5 -0.5  0.5 -0.5] * [1.]
      [-0.5  0.5 -0.5  0.5]   [1.]
      [ 0.5    -0.5    -0.1667  0.5   ]   [1.]
      [-0.5  0.5  0.5 -0.5]   [-1.]

    = [2.] = [2.][1.] = u2x2
      [-1.]        [-0.5]
      [-0.6667]        [-0.3333]
      [1.]        [0.5]
λ1 = 1/u1 = 1/[2.] =>[0.5]
Error = | ([0.5] - [2.])/[0.5] | => 3

-------------------------------------

Bx2 = [ 1.5 -0.5  0.5 -0.5] * [1.]
      [-0.5  0.5 -0.5  0.5